In [62]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re
import os
# from langchain.llms import OpenAI
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
import pickle
from transformers import pipeline








In [7]:
file_path = 'data.csv'
df = pd.read_csv(file_path)
print("Initial Data:")
display(df.head())

Initial Data:


,id,source,title,image,url,content,date,embedding,created_at,updated_at,summary
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698528/depo-pl...,"TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00,"[-0.01590039,-0.034130897,0.005732614,-0.01853...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Anggota Komisi VII DPR RI Rofik Hananto menyay...
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698522/jokowi-...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00,"[-0.017608976,-0.021786924,0.01547983,-0.00932...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Presiden Joko Widodo telah memerintahkan Wakil...
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698527/hnw-men...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00,"[0.00841488,-0.023665192,0.006762431,-0.013723...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698540/tim-dok...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00,"[-0.012671886,-0.0039057182,0.019575326,-0.016...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698536/bamsoet...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00,"[-0.015486176,-0.0125719,-0.0122843925,-0.0343...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Ketua MPR RI Bambang Soesatyo telah diangkat s...


In [ ]:
columns_to_drop = ['image', 'url', 'embedding', 'created_at', 'updated_at']
df.drop(columns=columns_to_drop, inplace=True)
print("\nAfter Dropping Unnecessary Columns:")
display(df.head())

In [ ]:
columns_rename_map = {
    'id': 'record_id',
    'source': 'news_source',
    'date': 'publication_date'
}
df.rename(columns=columns_rename_map, inplace=True)
print("\nAfter Renaming Columns:")
display(df.head())


In [ ]:
df.head()

In [ ]:
def roman_to_int(roman):
    roman_map = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 
        'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 
        'IX': 9, 'X': 10
    }

    return roman_map.get(roman, roman)


def convert_roman_to_int(text):
    def replace_roman(match):
        roman_numeral = match.group(0)
        return str(roman_to_int(roman_numeral))  # Convert Roman numeral to integer
    

    return re.sub(r'\b(I|II|III|IV|V|VI|VII|VIII|IX|X)\b', replace_roman, text)


for col in ['title', 'content', 'summary']:
    df[col] = df[col].apply(lambda x: convert_roman_to_int(x) if isinstance(x, str) else x)
    df[col] = df[col].str.lower()  # Convert to lowercase


display(df.head())


In [ ]:
df['Status'] = 1

print(df)

In [ ]:
cleaned_file_path = 'cleaned_data.csv'
df.to_csv(cleaned_file_path, index=False)
print(f"\nCleaned data saved to {cleaned_file_path}")
df.tail()

In [16]:

cleaned_file_path = 'cleaned_data.csv'
df = pd.read_csv(cleaned_file_path)

columns_to_tokenize = ['title', 'content', 'summary']

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

for column in columns_to_tokenize:
    df[f"{column}_tokens"] = df[column].apply(
        lambda x: tokenizer.tokenize(x) if pd.notnull(x) else []
    )
    df[f"{column}_token_ids"] = df[column].apply(
        lambda x: tokenizer.encode(x, truncation=True, max_length=512) if pd.notnull(x) else []
    )

tokenized_file_path = 'tokenized_data.csv'
df.to_csv(tokenized_file_path, index=False)
print(f"Tokenized data saved to {tokenized_file_path}")


Tokenized data saved to tokenized_data.csv


In [3]:
tokenized_df = pd.read_csv('tokenized_data.csv')
tokenized_df2= pd.read_csv('tokenized_data_hoax.csv')
df_merge = pd.merge(tokenized_df, tokenized_df2, on=['record_id', 'content','Status','title','publication_date','title_token_ids','title_tokens','content_tokens','content_token_ids'], how='outer')



In [16]:
df_merge.tail()


,record_id,news_source,title,content,publication_date,summary,Status,title_tokens,title_token_ids,content_tokens,content_token_ids,summary_tokens,summary_token_ids
36959,999287,,: kpk dilarang membawa brimob bersenjata masuk...,kpk dilarang membawa brimob bersenjata masuk g...,17-jan-16,,0,"[':', 'Ġk', 'p', 'k', 'Ġd', 'ilar', 'ang', 'Ġm...","[0, 35, 449, 642, 330, 385, 16813, 1097, 26012...","['k', 'p', 'k', 'Ġd', 'ilar', 'ang', 'Ġmem', '...","[0, 330, 642, 330, 385, 16813, 1097, 26012, 42...",,
36960,999472,,foto pejabat keuangan dibawah palu arit,jangan mau dialihkan kepada pakaian adat… foku...,20-aug-20,,0,"['f', 'oto', 'Ġpe', 'jab', 'at', 'Ġke', 'u', '...","[0, 506, 7087, 3723, 39805, 415, 7321, 257, 25...","['j', 'angan', 'Ġm', 'au', 'Ġdial', 'ih', 'kan...","[0, 267, 25750, 475, 1180, 11481, 4001, 11334,...",,
36961,999501,,gambar denny siregar musuh warga tasikmalaya d...,akhir nya bisa terkenal bang denny,17-aug-20,,0,"['g', 'amb', 'ar', 'Ġd', 'enny', 'Ġs', 'ire', ...","[0, 571, 3146, 271, 385, 11867, 579, 1885, 627...","['akh', 'ir', 'Ġn', 'ya', 'Ġb', 'isa', 'Ġter',...","[0, 7352, 853, 295, 2636, 741, 6619, 8470, 694...",,
36962,999848,,kaesang: bapak saya dengan kesederhaan bisa ni...,bapak saya dengan kesederhaan bisa nipu rakyat...,21-jul-20,,0,"['ka', 'es', 'ang', ':', 'Ġb', 'ap', 'ak', 'Ġs...","[0, 2348, 293, 1097, 35, 741, 1115, 677, 224, ...","['b', 'ap', 'ak', 'Ġsay', 'a', 'Ġden', 'gan', ...","[0, 428, 1115, 677, 224, 102, 3069, 3494, 449,...",,
36963,999860,,laser dari termometer gun akan merusak struktu...,"in: saya nolak. kalau anda mau periksa, bukan ...",20-jul-20,,0,"['l', 'aser', 'Ġd', 'ari', 'Ġterm', 'ometer', ...","[0, 462, 12425, 385, 1512, 1385, 12687, 1751, ...","['in', ':', 'Ġsay', 'a', 'Ġn', 'ol', 'ak', '.'...","[0, 179, 35, 224, 102, 295, 1168, 677, 4, 449,...",,


In [15]:
columns_to_fix = ['summary', 'summary_tokens', 'summary_token_ids','news_source']
df_merge[columns_to_fix] = df_merge[columns_to_fix].fillna('')


In [23]:
print(df_merge['news_source'].apply(type).value_counts())



news_source
<class 'str'>    36964
Name: count, dtype: int64


In [ ]:


model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

df_merge['combined_text'] = df_merge['title'] + " " + df_merge['content'] + " " + df_merge['summary']
df_merge['combined_text'] = df_merge['combined_text'].apply(lambda x: str(x) if pd.notna(x) else "")


text_embeddings = model.encode(df_merge['combined_text'].tolist(), batch_size=32, show_progress_bar=True)

np.save('embeddings.npy', text_embeddings)

print(text_embeddings.shape)


Batches:   0%|          | 0/1156 [00:00<?, ?it/s]

(36964, 384)


In [ ]:
os.environ["OPENAI_API_KEY"] = "Kalo mo pake masukin API key kalian bang"


In [ ]:
# df_merge['input'] = df_merge.apply(
#     lambda x: f"title: {' '.join(x['title_tokens'])} content: {' '.join(x['content_tokens'])} summary: {' '.join(x['summary_tokens'])}", axis=1
# )

# llm = OpenAI(model_name="gpt-4", temperature=0)

# prompt = PromptTemplate(
#     input_variables=["input"],
#     template="Analyze the following news and determine its authenticity: {input}"
# )

# llm_chain = LLMChain(llm=llm, prompt=prompt)

# def analyze_row(row):
#     try:
#         return llm_chain.run({"input": row['input']})
#     except Exception as e:
#         print(f"Error processing row: {e}")
#         return "Error"

# batch_size = 200

# for start in range(0, len(df_merge), batch_size):
#     batch = df_merge.iloc[start:start+batch_size]
#     batch['predicted_label'] = batch.apply(analyze_row, axis=1)
    
#     batch.to_csv(f"llm_predictions_batch_{start}.csv", index=False)
#     print(f"Processed batch {start}-{start + batch_size - 1}")

# print("Batch processing completed and saved.")

In [ ]:
embeddings = np.load("embeddings.npy").astype('float32')

index = faiss.IndexFlatL2(embeddings.shape[1])  

index.add(embeddings)

print(f"Index contains {index.ntotal} documents")

Index contains 36964 documents


In [53]:
embeddings = np.load("embeddings.npy").astype('float32')
documents = df_merge['content'].tolist()  # Document content
metadata = [{"title": row['title'], "summary": row['summary']} for _, row in df_merge.iterrows()]  # Metadata for documents

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

docstore_data = {str(i): {"page_content": documents[i], "metadata": metadata[i]} for i in range(len(documents))}
docstore = InMemoryDocstore(docstore_data)

index_to_docstore_id = {i: str(i) for i in range(len(documents))}

faiss.write_index(index, "faiss_index.index.faiss")

with open("docstore.pkl", "wb") as f:
    pickle.dump(docstore, f)
with open("index_to_docstore_id.pkl", "wb") as f:
    pickle.dump(index_to_docstore_id, f)

print("FAISS index, docstore, and mappings saved.")


FAISS index, docstore, and mappings saved.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

documents = []
with open("documents.txt", "r") as f:
    documents = f.readlines()

embeddings = np.load("document_embeddings.npy")
index = faiss.read_index("document_index.index")

def retrieve_similar_documents(query, top_k=3):
    query_embedding = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        query_output = model(**query_embedding)
    query_embedding = query_output.last_hidden_state.mean(dim=1).numpy()

    D, I = index.search(query_embedding, top_k)  
    similar_documents = [documents[idx] for idx in I[0]]
    return similar_documents

query = "AI technology is advancing rapidly, how is it affecting jobs?"
similar_docs = retrieve_similar_documents(query)

print("Top 3 similar documents:")
for doc in similar_docs:
    print(doc)

Top 3 similar documents:
Another document about machine learning.
This is a document about AI.

Another document about machine learning.


In [ ]:

fake_news_classifier = pipeline("text-classification", model="indobenchmark/indobert-base-p1")

def integrate_for_fake_news_detection(query, similar_documents):
    context = " ".join(similar_documents)  
    combined_input = f"Query: {query} Context: {context}"

    result = fake_news_classifier(combined_input)
    
    return result

result = integrate_for_fake_news_detection(query, similar_docs)
print("Fake News Detection Result:", result)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

c:\Users\Raja\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Raja\.cache\huggingface\hub\models--indobenchmark--indobert-base-p1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Fake News Detection Result: [{'label': 'LABEL_0', 'score': 0.2573208808898926}]
